In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from ChemModel import translator, quantum_net
from Arguments import Arguments
import random
import matplotlib.pyplot as plt
import csv, os
import pennylane as qml
from math import pi
import torch.nn as nn
import torch
import pickle

In [ ]:
def mask(net, positions = None):
    if positions:
         single = positions[0]
         two = positions[1]
         for i in single:
              net[i] = 'n'
         for i in two:
              net[12 + i] = 'n'
    else:
        mask = [random.sample(range(1, 12), 4), random.sample(range(12, 24), 6)]
        for i in range(2):
            for j in range(len(mask[i])):
                net[mask[i][j]] = 'n'
    return net

In [ ]:
net = [0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 3, 7, 6, 3, 1, 1, 10, 6, 5, 6, 6, 7]
net = mask(net, [[], [6, 10]])   # [single, two]
print(net)
design = translator(net)
weight = nn.Parameter(torch.rand(design['layer_repe'] * 12 * 2))
fig, _ = qml.draw_mpl(quantum_net, decimals=1, style="black_white", fontsize="x-small")(weight, design = design)
# plt.savefig('draw/net.jpg')

In [ ]:
import schemes_dist
schemes_dist.run(net, 500)

In [ ]:

filename = 'models/' + schemes_dist.net2str(net)
with open('data/OHhamiltonian', 'rb') as outfile:
    hamiltonian = pickle.load(outfile)

with open(filename, 'rb') as outfile:
    q_params = pickle.load(outfile)[1]
dev = qml.device("qiskit.aer", wires=12)
@qml.qnode(dev)
def cost_noise(theta):
    quantum_net(theta, design)
    return qml.expval(hamiltonian)    

print(cost_noise(q_params)) # execute the circuit once to generate the qasm
qasm = dev._circuit.qasm(formatted=True, filename = 'circuits/circuit_1Layer.qasm') # get the formatted qasm string